In [1]:
import codecs
import requests
from bs4 import BeautifulSoup
import random
from collections import OrderedDict

headers_of_request = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Accept-Encoding': 'gzip, deflate, sdch'
}

In [3]:
s = requests.Session()
s.headers.update(headers_of_request)
BaseUrl = 'http://www.wango.org'
SubUrl = '/resources.aspx?section=ngodir'

'''SubSubUrl = '&sub=region&regionID=5&col=cc3300'
'''

payload = OrderedDict([('sub','region'),('regionID', '5'),('col', 'cc3300')])
referer = {'Referer': 'http://www.wango.org/resources.aspx?section=ngodir'}
request_item = {
    "Countries": "ALL",
    "InterestAreas": "Environment",
    "currpage": 1,
    "keyword": "",
    "searching": "",
    "search_regionID": "",
    "search_country": "",
    "search_state": "",
    "search_mission": "",
    "search_focus": "",
    "search_city": "",
    "ffrom": "",
    "fto": "zz",
}

ngo_initial_response = requests.post(BaseUrl + SubUrl,params=payload,data=request_item,
                             headers=referer)
print ngo_initial_response.url
print ngo_initial_response.status_code

http://www.wango.org/resources.aspx?section=ngodir&sub=region&regionID=5&col=cc3300
200


In [5]:
ngo_soup = BeautifulSoup(ngo_initial_response.content,'lxml')
with open('ngo_response.txt','w') as output_file:
    output_file.write(ngo_soup.prettify().encode('utf-8'))

In [4]:
initial_content = ngo_soup.find('div',{'class': 'content'})
pages_content = initial_content.find('td',{'valign': 'top'}).find('div',{'align': 'left'})
number_of_pages = -1
print pages_content
for link in pages_content.findAll('a'):
    number_of_pages += 1
    
print number_of_pages

<div align="left">Page: <b>1</b>  <a href="javascript:goPage('2',  '', 'zz');void(0);">2</a>  <a href="javascript:goPage('3',  '', 'zz');void(0);">3</a>  <a href="javascript:goPage('4',  '', 'zz');void(0);">4</a>  <a href="javascript:goPage('5',  '', 'zz');void(0);">5</a>  <a href="javascript:goPage('6',  '', 'zz');void(0);">6</a>  <a href="javascript:goPage('7',  '', 'zz');void(0);">7</a>  <a href="javascript:goPage('8',  '', 'zz');void(0);">8</a>  <a href="javascript:goPage('9',  '', 'zz');void(0);">9</a>  <a href="javascript:goPage('10',  '', 'zz');void(0);">10</a>  <a href="javascript:goPage('11',  '', 'zz');void(0);">11</a>  <a href="javascript:goPage('12',  '', 'zz');void(0);">12</a>  <a href="javascript:goPage('13',  '', 'zz');void(0);">13</a>  <a href="javascript:goPage('14',  '', 'zz');void(0);">14</a>  <a href="javascript:goPage('14',  '', 'zz');void(0);">&gt;</a>  <a href="javascript:goPage('14',  '', 'zz');void(0);">&gt;&gt;</a>  </div>
14
